# Tome Notebook

This notebook is used for experimenting with LLMs to support my Tome App. 

In [1]:
import pandas as pd
import time
from datetime import datetime
import warnings
from IPython.display import clear_output

import boto3
from botocore.exceptions import ClientError
import json

client = boto3.client("bedrock-runtime", region_name="eu-west-1")

pd.set_option('display.max_columns', None)

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=UserWarning)

In [2]:
class Claude: 

    model_id = 'eu.anthropic.claude-3-5-sonnet-20240620-v1:0'

    def invoke(self, prompt: str): 
        # Start a conversation with the user message.
        
        conversation = [
            {
                "role": "user",
                "content": [{"text": prompt}],
            }
        ]
        
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=self.model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": 2000, "temperature": 0, "topP": 0.9},
            )
        
            # Extract and print the response text.
            return response["output"]["message"]["content"][0]["text"]
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            exit(1)

## Generating Questions
This part of the notebook will use an LLM to generate a series of max $N_Q$ questions on a given Knowledge Base (KB).

The LLM will be given the KB as contextual information and asked to generate $N_Q$ questions on that. 

**Notes**<br>
* Adding the following in the prompt helped a lot in getting more interesting, open-ended questions: *Questions should require a bit of elaboration, not just a few words as an answer.*

In [5]:
class QuestionsGenerator: 

    model_id = 'eu.anthropic.claude-3-5-sonnet-20240620-v1:0'

    def generate_questions(self, num_questions: int = 10): 
        """Generates a list of questions

        Params
        ----
        - kb a string containing the knowledge base to generate questions on

        Returns
        ----
        - a list of questions
        """
        # 1. Load the context
        kb_file_path = './data/tome/kb_cortes_mexico.txt'

        with open(kb_file_path, 'r', encoding='utf-8') as file:
            kb = file.read()

        # 2. Define the System Prompt
        system_prompt = f"""
        You are acting as a Quiz's question generating engine. Your role is, given a knowledge base (hereafter KB) to generate 10 questions based on the content of KB. 
        The questions CAN ONLY REFER to the content of KB. 
        The following is the KB that is given to you: 
        ----------------
        {kb}
        ----------------
        Generate {num_questions} questions based on the KB. 
        Questions should require a bit of elaboration, not just a few words as an answer. 
        Provide the questions as a JSON object with only one field called questions which will be an array of strings.
        Do not provide anything else. Only provide a JSON object. No other text.
        """

        conversation = [
            {
                "role": "user", 
                "content": [{"text": system_prompt}]
            },
        ]
        
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=self.model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": 2000, "temperature": 0, "topP": 0.9},
            )
        
            # Extract and print the response text.
            return response["output"]["message"]["content"][0]["text"]
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            exit(1)

In [36]:
start_time = time.time()

questions_text = QuestionsGenerator().generate_questions()

end_time = time.time()

question_gen_execution_time = end_time - start_time

In [37]:
print(f"Questions generated in {question_gen_execution_time:.2f} seconds\n")
for q in json.loads(questions_text)['questions']:
    print(" - " + q)

Questions generated in 7.00 seconds

 - Describe the circumstances surrounding Cortes' hasty departure from Cuba and his initial journey to Cozumel.
 - Explain the significance of Malinche in Cortes' expedition and what her role might have been.
 - How did Cortes manage to form an alliance with the Totonacs, and what strategy did he use to solidify this relationship?
 - Discuss the reasons behind Cortes' decision to found Vera Cruz and how this action helped legitimize his position.
 - Describe the challenges Cortes faced in Tlaxcala and how the outcome of this conflict benefited his campaign.
 - Explain the events that led to Cortes capturing Montezuma and ruling through him for seven months.
 - How did Cortes deal with the threat posed by Narvaez, who was sent by Velazquez with 1500 men?
 - Describe the events of 'La noche triste' and its impact on Cortes' expedition.
 - Explain the desperate battle Cortes fought against 100,000 Aztecs and the unconventional tactic he used to secure 

## Rating Answers

This section uses the LLM to rate a user's answers. 

My notes: 
* Generating everything with a single prompt was complicated, so I split it in two prompts:
    * One that would give a Chain of Thought prompt for the rating to fit what I wanted
    * One that would use the answer to just extract the rating and explanation as a json

In [58]:
class RatingAgent: 

    model_id = 'eu.anthropic.claude-3-5-sonnet-20240620-v1:0'

    def rate_answer(self, question: str, answer: str): 

        # 1. Load the context
        kb_file_path = './data/tome/kb_cortes_mexico.txt'

        with open(kb_file_path, 'r', encoding='utf-8') as file:
            kb = file.read()

        # 2. Define the First Prompt
        system_prompt = f"""
        You are a Quiz engine and you have previously generated some questions based on a knowledge base. 
        You are going to be provided a question (that you generated) and the user's answer to that question. 
        You are asked to rate the answer on a scale from 1 to 5, 1 being the lowest score, 5 the highest. 
        This is the KNOWLEDGE BASE:
        ----------------
        {kb}
        ----------------
        This is the QUESTION: 
        ----------------
        {question}
        ----------------
        This is the user's ANSWER: 
        ----------------
        {answer}
        ----------------
        Rate the answer as a float with maximum one decimal number.
        You MUST only use the knowledge base to rate the answer. 
        If some information is provided in the answer and cannot be found in the knowledge base, ignore it and do not mention it in your explanations. 

        To rate the answer you MUST perform the following steps: 
        1. List the most important aspects as a short list
        2. Check how many of those aspects are covered by the answer
        3. Rate the answer
            - if the answer misses half or more main aspects, it should NOT get a rating higher than 2
            - if the answer gets all the important aspects it should get a rating of 5. 
            - Minor omissions must be ignored. 
        """

        conversation = [
            {
                "role": "user", 
                "content": [{"text": system_prompt}]
            },
        ]
        
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=self.model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": 2000, "temperature": 0.2, "topP": 0.9},
            )
        
            # Extract and print the response text.
            return response["output"]["message"]["content"][0]["text"]
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            exit(1)

In [55]:
class FormattingAgent: 
    
    model_id = 'eu.anthropic.claude-3-5-sonnet-20240620-v1:0'

    def format_rating(self, rating: str): 
    
        # Define the First Prompt
        system_prompt = f"""
        You are a JSON formatter with a brain. 
        You are being given some text that rates a user answer to a question in the context of a Quiz. In that text there two VERY IMPORTANT things that I need: 
        1. A rating, expressed as either an integer or a float with a single decimal (from 1 to 5)
        2. An explanation of the rating.
        You are asked to extract these two pieces of information and provide them in a JSON format. 
        This is the text from which you must extract that information:
        ----------------
        {rating}
        ----------------

        Provide the rating in a JSON format. You must provide at least the following fields:
        - rating which will contain the rating value as a float
        - explanation which will contain the explanations for the rating, with corrections of what the user got wrong. Be synthetic. 

        ONLY provide the answer in a JSON format. Do not provide additional text. 
        """

        conversation = [
            {
                "role": "user", 
                "content": [{"text": system_prompt}]
            },
        ]
        
        try:
            # Send the message to the model, using a basic inference configuration.
            response = client.converse(
                modelId=self.model_id,
                messages=conversation,
                inferenceConfig={"maxTokens": 2000, "temperature": 0, "topP": 0.9},
            )
        
            # Extract and print the response text.
            return response["output"]["message"]["content"][0]["text"]
        
        except (ClientError, Exception) as e:
            print(f"ERROR: Can't invoke '{self.model_id}'. Reason: {e}")
            exit(1)

In [59]:
start_time = time.time()

q = "Describe Cortes' journey from Cuba to Cozumel and the challenges he faced along the way."
a = f"""
Cortes left from Cuba in early 1519. He had to escale silently from Cuba, since the governor was actually trying to block cortes from leaving. 
There were no challenges to travel to Cozumel. The challenges started in Cozumel. 
"""

# Rate
rating_text = RatingAgent().rate_answer(q, a)

intermediate_time = time.time()

# Format
rating = FormattingAgent().format_rating(rating_text)

end_time = time.time()

exec_time = end_time - start_time

print(f"Answer of the Rating Agent ({intermediate_time - start_time} seconds):")
print(rating_text)
print(f"Answer of the Formatting Agent ({exec_time} seconds)")
print(rating)

Answer of the Rating Agent (7.058557033538818 seconds):
Let's evaluate the answer based on the knowledge base provided:

1. Most important aspects:
   - Cortes leaves Cuba in haste
   - Velasquez's hesitation about sending Cortes
   - Fleet heading towards Cozumel
   - Goal to find naufragees living with natives
   - Fleet scattered by bad weather
   - Alvarado arrives first, city deserted

2. Aspects covered by the answer:
   - Cortes left from Cuba in early 1519
   - He had to leave silently due to the governor trying to block him

3. Rating the answer:
   The answer covers only one of the six main aspects listed (Cortes leaving Cuba hastily). It misses important details about the journey to Cozumel, the purpose of going there, and what happened upon arrival. More than half of the main aspects are missing.

   Given these considerations, I would rate this answer as 1.5 out of 5.

   The answer provides very limited information from the knowledge base and misses most of the key points

In [60]:
start_time = time.time()

q = "Describe Cortes' journey from Cuba to Cozumel and the challenges he faced along the way."
a = f"""
Cortes left from Cuba in early 1519. He had to escale silently from Cuba, since the governor was actually trying to block cortes from leaving, because he had last-minute hesitations. 
There was bad weather during the travel which scattered the fleet. This meant that Alvarado got there first and having a violent nature he attacked the Indians he encountered, scaring them in leaving Cozumel. 
Cortes wanted to find some naufragees from the previous explorative travels. He would use them as translators to be able to communicate with the natives. 
Aguilar, who had been enslaved by the Maya, 20241117-trainingExamples-cleaned.jsonwas the naufragee that was found in Cozumel. He joined Cortes for the rest of his adventure.
"""

# Rate
rating_text = RatingAgent().rate_answer(q, a)

intermediate_time = time.time()

# Format
rating = FormattingAgent().format_rating(rating_text)

end_time = time.time()

exec_time = end_time - start_time

print(f"Answer of the Rating Agent ({intermediate_time - start_time} seconds):")
print(rating_text)
print("-------------------------------------")
print(f"Answer of the Formatting Agent ({exec_time} seconds)")
print(rating)

Answer of the Rating Agent (7.230735540390015 seconds):
Let's follow the steps to rate this answer:

1. Most important aspects from the knowledge base:
   - Cortes leaves Cuba in haste due to Velasquez's hesitations
   - Fleet heads towards Cozumel
   - Goal is to find naufragees living with natives
   - Fleet scattered by bad weather
   - Alvarado arrives first, city deserted due to his conquering approach
   - They get Aguilar, a naufragee enslaved by Maya, to join them

2. Aspects covered by the answer:
   - Cortes leaving Cuba hastily due to governor's hesitations
   - Bad weather scattering the fleet
   - Alvarado arriving first and scaring off natives
   - Goal of finding naufragees as translators
   - Aguilar, enslaved by Maya, found and joined Cortes

3. Rating the answer:
   The answer covers all the main aspects mentioned in the knowledge base. It accurately describes Cortes' departure from Cuba, the challenges faced during the journey, and the key events upon reaching Cozume